# Pairwise Global Alignment

Goal is to create a set of pairwise global alignments based on the mixed sampels with the correct references.

-   Align all sequences to the 2 possible references (global alignment with muscle)
-   Use Biopython `alignment.counts()` to summarise the following:
    -   Number of identities
    -   Number of mismatches
    -   Number of gaps


In [1]:
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio import SeqIO, AlignIO
import pandas as pd
import os
import sys
import subprocess
import re

In [2]:
pip show biopython

Name: biopython
Version: 1.85
Summary: Freely available tools for computational molecular biology.
Home-page: https://biopython.org/
Author: The Biopython Contributors
Author-email: biopython@biopython.org
License: 
Location: /Users/joonklaps/opt/anaconda3/envs/lasvdedup-env/lib/python3.12/site-packages
Requires: numpy
Required-by: lasv-dedup, lasvdedup, nwkfmt
Note: you may need to restart the kernel to use updated packages.


In [3]:
got_dict = {
    "Eddard": ["MN090188.1", "MN090277.1"],
    "Catelyn": ["MN090188.1", "MN090277.1"],
    "Robb": ["MN090188.1", "MN090277.1"],
    "Jon": ["MN090240.1", "MN090277.1"],
    "Sansa": ["MN090240.1", "MN090277.1"],
    "Arya": ["MN090240.1", "MN090277.1"],
    "Daenerys": ["MZ766668.1", "MN090277.1"],
    "Tyrion": ["MZ766668.1", "MN090277.1"],
    "Jaime": ["MZ766668.1", "MN090277.1"],
    "Bran": ["MN090277.1"],
    "Rickon": ["MN090188.1"],
    "Theon": ["MN090240.1"],
    "Jorah": ["MZ766668.1"],
}

skiphybrid_ri = SeqIO.to_dict(SeqIO.parse("./data/hybrid-vs-skiphybrid/skiphybrid.unique.fasta", "fasta"))
noskiphybrid_ri = SeqIO.to_dict(SeqIO.parse("./data/hybrid-vs-skiphybrid/skiphybrid.unique.fasta", "fasta"))
consensus_ri = SeqIO.to_dict(SeqIO.parse("./data/influence-reference/unique.fasta", "fasta"))
reference_ri = SeqIO.to_dict(SeqIO.parse("./data/ref.unique.fasta", "fasta"))

In [ ]:
def align_with_mafft(input, output):
    # mafft_path = "/Users/joonklaps/opt/anaconda3/bin/mafft"
    mafft_path = "/opt/conda/bin/mafft"  # `which mafft` should return this path
    cline = f"{mafft_path} --globalpair --maxiterate 1000 --adjustdirection --thread -1 {input} > {output}"
    prog = subprocess.run(cline, shell=True, check=True, stderr=subprocess.PIPE)

    if os.path.isfile(output):
        print(f"Alignment successful: {output}")
    else:
        print("Alignment failed, output file does not exist.")
        print(f"Command: {cline}")
        print(f"Error: {prog.stderr.decode()}")
        sys.exit(1)


def generate_alignment(dir, record, reference):
    # Create alignments directory if it doesn't exist
    os.makedirs(dir, exist_ok=True)

    fasta = dir + f"/{record.id}.vs.{reference.id}.fasta".replace("-", ".").replace("|", "_")
    alignment = dir + f"/{record.id}.vs.{reference.id}.aln".replace("-", ".").replace("|", "_")

    if os.path.isfile(alignment):
        with open(alignment) as f:
            content = f.read()
            if content.count(">") == 2:
                return alignment

    if not os.path.isfile(fasta):
        with open(fasta, "w") as f:
            f.write(f">{record.id}\n{record.seq}\n>{reference.id}\n{reference.seq}\n")

    align_with_mafft(fasta, alignment)
    return alignment

In [ ]:
def generate_pairwise_alignments(target_records, reference_records, got_mapping, output_dir):
    """
    Generate pairwise alignments between target records and their corresponding reference sequences.

    Parameters:
    -----------
    target_records : dict
        Dictionary of target sequences to align (SeqRecord objects)
    reference_records : dict
        Dictionary of reference sequences (SeqRecord objects)
    got_mapping : dict
        Dictionary mapping GOT characters to reference IDs
    output_dir : str
        Directory to store the alignment files

    Returns:
    --------
    dict
        Dictionary of alignment statistics for each target-reference pair
    """
    result_dict = {}
    for _, record in target_records.items():
        parts = record.id.split("-")
        if len(parts) >= 2:
            got = re.match(r"([A-Za-z]+)", parts[0]).group(1)
            try:
                other = "-".join(parts[1:])
                if not "it2" in other:
                    continue

                db = re.match(r"((M[NZ][0-9\.]+(\-M[NZ][0-9\.]+)?)|virosaurus|reference)", other).group(1)
            except AttributeError as e:
                print(f"Skipping {record.id} due to invalid ID format")
                raise ValueError(f"Invalid ID format for {record.id}", e)

            # Skip if got is not in mapping
            if got not in got_mapping:
                print(f"No reference mapping found for {got}")
                continue

            for refid in got_mapping[got]:
                print(f"{record.id} vs {refid} ...")
                reference = reference_records.get(refid)

                # Skip if reference not found
                if reference is None:
                    print(f"Reference {refid} not found")
                    continue

                alignment_path = generate_alignment(output_dir, record, reference)

                if alignment_path is None:
                    print(f"      Skipping {record.id} vs {refid} due to alignment failure")
                    continue

                try:
                    pwa = AlignIO.read(alignment_path, "fasta", 2)
                    counts = pwa.alignment.counts()._asdict()
                    result_dict[(record.id, refid)] = {
                        **counts,
                        "query_length": len(pwa[0].seq),  # Length of the first sequence
                        "number_of_gaps": pwa[0].count("-"),  # Number of gaps in the first sequence
                        "number_of_Ns": pwa[0].count("n") + pwa[0].count("N"),  # Number of Ns in the first sequence
                        "alignment_length": pwa.alignment.length,
                        "got": got,
                        "db": db,
                        "alignment_path": alignment_path,
                    }
                except Exception as e:
                    print(f"Error processing alignment for {record.id} vs {refid}")
                    print(f"Error: {str(e)}")
        else:
            print(f"Skipping {record.id} due to invalid ID format")

    return result_dict

In [10]:
skip_hybrid_dict = generate_pairwise_alignments(
    target_records=skiphybrid_ri,
    reference_records=reference_ri,
    got_mapping=got_dict,
    output_dir="./data/hybrid-vs-skiphybrid/skiphybrid-vs-ref/",
)

Arya-MN090240_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-MN090240_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-MN090277-MN090240_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-MN090277-MN090240_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-MN090277_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-MN090277_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-reference_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-reference_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-reference_cl1_it2.consensus_ivar vs MN090240.1 ...
Arya-reference_cl1_it2.consensus_ivar vs MN090277.1 ...
Arya-virosaurus_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-virosaurus_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-virosaurus_cl1_it2.consensus_ivar vs MN090240.1 ...
Arya-virosaurus_cl1_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090188_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090240_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090277-MN090188_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090277-MN090240_cl0

In [11]:
noskip_hybrid_dict = generate_pairwise_alignments(
    target_records=noskiphybrid_ri,
    reference_records=reference_ri,
    got_mapping=got_dict,
    output_dir="./data/hybrid-vs-skiphybrid/hybrid-vs-ref/",
)

Arya-MN090240_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-MN090240_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-MN090277-MN090240_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-MN090277-MN090240_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-MN090277_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-MN090277_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-reference_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-reference_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-reference_cl1_it2.consensus_ivar vs MN090240.1 ...
Arya-reference_cl1_it2.consensus_ivar vs MN090277.1 ...
Arya-virosaurus_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-virosaurus_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-virosaurus_cl1_it2.consensus_ivar vs MN090240.1 ...
Arya-virosaurus_cl1_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090188_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090240_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090277-MN090188_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090277-MN090240_cl0

In [12]:
influence_ref_dict = generate_pairwise_alignments(
    target_records=consensus_ri,
    reference_records=reference_ri,
    got_mapping=got_dict,
    output_dir="./data/influence-reference/alignment-vs-ref/",
)

Arya-MN090240_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-MN090240_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-MN090277-MN090240_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-MN090277-MN090240_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-MN090277_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-MN090277_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-reference_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-reference_cl0_it2.consensus_ivar vs MN090277.1 ...
Arya-reference_cl1_it2.consensus_ivar vs MN090240.1 ...
Arya-reference_cl1_it2.consensus_ivar vs MN090277.1 ...
Arya-virosaurus_cl0_it2.consensus_ivar vs MN090240.1 ...
Arya-virosaurus_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090188_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090240_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090277-MN090188_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090277-MN090240_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN090277-MZ766668_cl0_it2.consensus_ivar vs MN090277.1 ...
Bran-MN09027

In [ ]:
pd.DataFrame.from_dict(noskip_hybrid_dict, orient="index").to_csv("./data/hybrid-vs-skiphybrid/hybrid-vs-ref.csv")
pd.DataFrame.from_dict(skip_hybrid_dict, orient="index").to_csv("./data/hybrid-vs-skiphybrid/skiphybrid-vs-ref.csv")
pd.DataFrame.from_dict(influence_ref_dict, orient="index").to_csv("./data/influence-reference/consensus-vs-ref.csv")